In [3]:
import pandas as pd
import numpy as np
import os

In [13]:
data = {}
root = 'caterpillar_data'
for file in os.listdir(root):
    name, num = file.split('.')[0].split('_')
    file_path = os.path.join(root, file)
    df = pd.read_hdf(file_path, key='star')
    
    df.sort_values('idstar', inplace=True, kind='stable')
    prev_val = -1
    prev_id = 0
    df['cluster_id'] = 0
    cluster_ids = np.zeros((len(df)), dtype=np.int32)
    for i, (idx, row) in enumerate(df.iterrows()):
        if (row['idstar'] - prev_val)>0.1:
            prev_id+=1
            prev_val = row['idstar']
        cluster_ids[i] = prev_id
    df['cluster_id'] = cluster_ids
    print(prev_id)
    if name not in data.keys():
        data[name] = {}
    data[name][num] = df
    

81
1
2
43
10
1
1
77
2
10
0
6
2
1
3
178
16
4
11
68
66
2
1
1
22
97
16
2
2
1
7
121
2
0
1
2
13
69
0
1
5
11
1
2
1
17
144
0
3
2
21
2
4
14
1
89
25
1
1
4
177
1
61
4
146
6
0
13
1
109
13
4
2
93
1
2
3
1
0
127
1
2
72
2
120
0
17
2
0
10
1
3
1
2
63
1
1
1
10
19
11
2
201
2
100
1
7
4
133
81
101
19
4
1
1
11
2
2
8
0
16
3
7
106
110
1
3
1
3
0
1
15
112
1
98
2
3
2
22
0
4
2
2
17
1
2
163
1
4
107
0
145
2
0
11
3
10
2
3
0


In [45]:
for key in data.keys():
    max_id = 0
    data[key]['all'] = pd.DataFrame(columns = data[key]['0'].columns)
    data[key]['all']['cluster_id'] = data[key]['all']['cluster_id'].astype('int32')
    for i in range(0,5):
        df = data[key][str(i)].copy()
        df['cluster_id'] += max_id
        if len(df)>0:
            data[key]['all'] = pd.concat([data[key]['all'], df])
        max_id = data[key]['all']['cluster_id'].max()
    print(data[key]['0']['cluster_id'].max(), data[key]['1']['cluster_id'].max(), data[key]['2']['cluster_id'].max(),\
          data[key]['3']['cluster_id'].max(), data[key]['4']['cluster_id'].max(), data[key]['all']['cluster_id'].max())

81 16 2 1 nan 100
77 11 1 2 1 92
163 22 2 2 nan 189
43 7 4 1 1 56
121 10 4 2 1 138
178 22 3 nan 1 204
107 17 nan 1 2 127
145 15 4 2 2 168
68 10 nan nan 2 80
66 6 3 1 1 77
112 11 2 2 nan 127
98 10 3 1 2 114
97 16 3 2 2 120
144 25 2 2 1 174
63 7 2 1 2 75
109 13 1 2 1 126
69 13 1 4 nan 87
133 10 4 nan 3 150
81 19 2 1 nan 103
201 19 5 1 1 227
93 11 4 2 nan 110
101 11 2 1 1 116
89 13 3 3 2 110
100 11 1 3 nan 115
127 17 1 2 1 148
72 8 3 1 1 85
120 16 2 4 1 143
177 21 3 1 nan 202
110 10 4 2 2 128
61 4 1 2 1 69
146 14 7 nan 1 168
106 17 6 1 1 131


In [46]:
print(data['1079897']['all'].dtypes)

jzstar        float64
jrstar        float64
jphistar      float64
vstar         float64
idstar        float64
estar         float64
cluster_id      int32
dtype: object


In [47]:
output = 'labeled_caterpillar_data'
for key in data.keys():
    for key2 in data[key].keys():
        data[key][key2].to_hdf(os.path.join(output, f'labeled_{key}_{key2}.h5'), key='star')